In [1]:
import os
import sys
import json
import shutil
import numpy as np
import pandas as pd

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

In [3]:
columns = read_json('../../orderedListGenres.json')
print(len(columns), columns)

20 ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']


In [ ]:
# 模型輸出儲存的檔案
# SAVE_NAME = 'resnet50_trainable_8e-5_starwars'
HISTORY_SAVE = './history/' + SAVE_NAME + '.csv'
WEIGHTS_SAVE = './weight/' + SAVE_NAME + '.h5'

# ResNet50 Model

In [4]:
# read dataframe
train_df = pd.read_csv('../filename2genreMat.csv')
test_df = pd.read_csv('../poster2genreMat.csv')
print(train_df.shape, test_df.shape)

(37959, 22) (165, 22)


In [5]:
valid_df = train_df.sample(frac = 0.2, random_state = 42)
train_df = train_df.drop(valid_df.index)
print(train_df.shape, valid_df.shape)

(30367, 22) (7592, 22)


In [6]:
from keras.applications import ResNet50
from keras import models
from keras.models import Model
from keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator
from keras import layers
from keras import regularizers, optimizers
# from keras.applications.model import ResNet152

Using TensorFlow backend.


In [7]:
# 資料路徑
DATASET_PATH  = '../IGimg'

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 20

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 32

# Epoch 數
NUM_EPOCHS = 100

In [8]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(dataframe=train_df,
                                            directory=DATASET_PATH,
                                            x_col="filename",
                                            y_col=columns,
                                            batch_size=BATCH_SIZE,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="other",
                                            target_size=IMAGE_SIZE)

valid_generator=test_datagen.flow_from_dataframe(dataframe=valid_df,
                                                 directory=DATASET_PATH,
                                                 x_col="filename",
                                                 y_col=columns,
                                                 batch_size=BATCH_SIZE,
                                                 seed=42,
                                                 shuffle=True,
                                                 class_mode="other",
                                                 target_size=IMAGE_SIZE)

Found 30367 validated image filenames.
Found 7592 validated image filenames.


# Fixed the last Layer from 1000 classes into N classes

In [ ]:
conv_base = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
conv_base.summary()

In [ ]:
model = models.Sequential()
model.add(conv_base)        # 將卷積基底視為層加入 Sequential 模型中
model.add(layers.AveragePooling2D(pool_size=(7, 7)))
model.add(layers.Flatten()) # 攤平
model.add(layers.Dense(NUM_CLASSES, activation='sigmoid')) # 增加密集層分類器
model.summary() # 查看模型摘要

In [ ]:
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = True

print('This is the number of trainable weights after freezing the conv base:', len(model.trainable_weights))

In [ ]:
model.compile(optimizer=Adam(lr=8e-5),
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

# Training Model

In [ ]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

In [ ]:
# 訓練模型
from keras.callbacks import EarlyStopping
callbacks = [EarlyStopping(monitor='val_loss')]

history = model.fit_generator(generator = train_generator,
                              steps_per_epoch = STEP_SIZE_TRAIN,
                              validation_data = valid_generator,
                              validation_steps = STEP_SIZE_VALID,
                              epochs = NUM_EPOCHS,
                              callbacks=callbacks)

In [ ]:
# 儲存訓練好的模型
model.save(WEIGHTS_SAVE)

In [ ]:
# save history
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

his_df = pd.DataFrame(data = {
    'Epoch' : epochs,
    'Loss' : loss,
    'Acc' : acc,
    'Val_loss' : val_loss,
    'Val_acc' : val_acc
})
his_df = his_df[['Epoch', 'Loss', 'Acc', 'Val_loss', 'Val_acc']]
his_df.to_csv(HISTORY_SAVE, index = False)

# Plot acc & loss

In [ ]:
# 繪製結果
import matplotlib.pyplot as plt

plt.figure()

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.show()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Testing

### Testing set: Poster

In [ ]:
SAVE_NAME = SAVE_NAME + '_poster'

In [ ]:
test_df = pd.read_csv('../poster2genreMat.csv')
test_df

In [ ]:
DATASET_PATH = '../Poster'

In [ ]:
test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                                directory=DATASET_PATH,
                                                x_col="filename",
                                                batch_size=1,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=IMAGE_SIZE)

In [ ]:
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                             steps=STEP_SIZE_TEST,
                             verbose=1)

# Output

In [ ]:
pred.shape

In [ ]:
predictions = pred

results=pd.DataFrame(predictions, columns=columns)
results["filename"] = test_generator.filenames
ordered_cols = ["filename"] + columns
results = results[ordered_cols] #To get the same column order
results.to_csv("./output/"+ SAVE_NAME + "_result.csv",index=False)

## Label top n dynamic

In [ ]:
true_df = test_df

In [ ]:
results = pd.read_csv("./output/"+ SAVE_NAME + "_result.csv")
results

In [ ]:
labels = read_json('../../genresDic.json')

In [ ]:
def countAcc(predList, trueList):
    fraction = 0
    denominator = len(trueList)
    for g in predList:
        if g in trueList:
            fraction += 1 
    return fraction / denominator

In [ ]:
li = []
for i, row in true_df.iterrows():
    username = row['filename']
    if username == results.loc[i,:]['filename']: # username equals
        print(i, username)
        accname = username.split('.')[0]
        truth = labels[accname]
        numTrue = sum(row[2:])
        print('count:', numTrue)
        
        s = results.loc[i,:][1:]
        top_n = s.sort_values(ascending=False)[:numTrue]
        inList = list(top_n.index)
        
        onehot = []
        for g in columns:
            onehot.append(int(g in inList))
            
        accu = countAcc(inList, truth)
        print(accu, inList, truth)
        
        appList = [username, accu, truth, inList] + onehot
        li.append(appList)
        

In [ ]:
acc_df = pd.DataFrame(li, columns = ['username', 'acc', 'truth', 'top_n'] + columns)
acc_df

In [ ]:
acc_df.to_csv("./output/"+ SAVE_NAME + "_bi.csv", index = 0)

In [ ]:
acc_df['acc'].mean()

# Feature Extraction
## 測試reload model準確度

In [14]:
DATASET_PATH  = '../IGimg'
train_df = pd.read_csv('../filename2genreMat.csv')
print(train_df.shape)
train_df

(37959, 22)


,from,filename,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,../Posts_165/21bridgesmovie/217501830640240439...,21bridgesmovie_22217.jpg,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,../Posts_165/21bridgesmovie/217791736649498418...,21bridgesmovie_22218.jpg,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,../Posts_165/21bridgesmovie/216903847292888510...,21bridgesmovie_22219.jpg,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,../Posts_165/21bridgesmovie/219893585468032379...,21bridgesmovie_22220.jpg,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,../Posts_165/21bridgesmovie/218219116847723486...,21bridgesmovie_22221.jpg,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37954,../Posts_165/zombieland/2137359683396335880_91...,zombieland_7839.jpg,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
37955,../Posts_165/zombieland/2137359683396335880_91...,zombieland_7840.jpg,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
37956,../Posts_165/zombieland/2137359683396335880_91...,zombieland_7841.jpg,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
37957,../Posts_165/zombieland/2218432031066716113_91...,zombieland_7842.jpg,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [13]:
# SAVE_NAME = 'test'
# DATASET_PATH = '../Poster'

# test_df = pd.read_csv('../poster2genreMat.csv')
# print(test_df.shape)
# test_df

(165, 22)


,from,filename,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,History,Horror,Music,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,../Profiles_165/21bridgesmovie/poster.jpeg,21bridgesmovie.jpg,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,../Profiles_165/47metersdown/poster.jpeg,47metersdown.jpg,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,../Profiles_165/abeautifuldaymovie/poster.jpeg,abeautifuldaymovie.jpg,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,../Profiles_165/abominablemovie/poster.jpeg,abominablemovie.jpg,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,../Profiles_165/adastramovie/poster.jpeg,adastramovie.jpg,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,../Profiles_165/wrinklestheclown/poster.jpeg,wrinklestheclown.jpg,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
161,../Profiles_165/xmenmovies/poster.jpeg,xmenmovies.jpg,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
162,../Profiles_165/yardiefilm/poster.jpeg,yardiefilm.jpg,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
163,../Profiles_165/yesterdaymovie/poster.jpeg,yesterdaymovie.jpg,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [11]:
WEIGHTS_LOAD = './weight/resnet50_trainable_9e-5_newdata.h5'

from keras.models import load_model

model = load_model(WEIGHTS_LOAD)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               25690368  
_________________________________________________________________
dense_4 (Dense)              (None, 20)                5140      
Total params: 49,283,220
Trainable params: 49,230,100
Non-trainable params: 53,120
_________________________________________________________________


In [15]:
test_generator=test_datagen.flow_from_dataframe(dataframe=train_df, #test_df,
                                                directory=DATASET_PATH,
                                                x_col="filename",
                                                batch_size=1,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=IMAGE_SIZE)

Found 37959 validated image filenames.


In [16]:
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size

In [17]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                             steps=STEP_SIZE_TEST,
                             verbose=1)

37959/37959 [==============================] - 319s 8ms/step


In [18]:
pred.shape

(37959, 20)

In [19]:
SAVE_NAME = 'resnet50_trainable_9e-5_newdata_IGphoto'

In [20]:
predictions = pred

results=pd.DataFrame(predictions, columns=columns)
results["filename"] = test_generator.filenames
ordered_cols = ["filename"] + columns
results = results[ordered_cols] #To get the same column order
results.to_csv("./output/"+ SAVE_NAME + "_result.csv",index=False)

In [21]:
true_df = train_df#test_df
labels = read_json('../../genresDic.json')

In [22]:
def countAcc(predList, trueList):
    fraction = 0
    denominator = len(trueList)
    for g in predList:
        if g in trueList:
            fraction += 1 
    return fraction / denominator

In [24]:
li = []
for i, row in true_df.iterrows():
    username = row['filename']
    if username == results.loc[i,:]['filename']: # username equals
        print(i, username)
        accname = username.split('_')[0]#username.split('.')[0]
        truth = labels[accname]
        numTrue = sum(row[2:])
        print('count:', numTrue)
        
        s = results.loc[i,:][1:]
        top_n = s.sort_values(ascending=False)[:numTrue]
        inList = list(top_n.index)
        
        onehot = []
        for g in columns:
            onehot.append(int(g in inList))
            
        accu = countAcc(inList, truth)
        print(accu, inList, truth)
        
        appList = [username, accu, truth, inList] + onehot
        li.append(appList)

0 21bridgesmovie_22217.jpg
count: 3
1.0 ['Drama', 'Crime', 'Action'] ['Action', 'Crime', 'Drama']
1 21bridgesmovie_22218.jpg
count: 3
1.0 ['Crime', 'Action', 'Drama'] ['Action', 'Crime', 'Drama']
2 21bridgesmovie_22219.jpg
count: 3
1.0 ['Drama', 'Crime', 'Action'] ['Action', 'Crime', 'Drama']
3 21bridgesmovie_22220.jpg
count: 3
1.0 ['Drama', 'Crime', 'Action'] ['Action', 'Crime', 'Drama']
4 21bridgesmovie_22221.jpg
count: 3
1.0 ['Drama', 'Crime', 'Action'] ['Action', 'Crime', 'Drama']
5 21bridgesmovie_22222.jpg
count: 3
1.0 ['Crime', 'Action', 'Drama'] ['Action', 'Crime', 'Drama']
6 21bridgesmovie_22223.jpg
count: 3
0.6666666666666666 ['Drama', 'Action', 'Horror'] ['Action', 'Crime', 'Drama']
7 21bridgesmovie_22224.jpg
count: 3
1.0 ['Drama', 'Action', 'Crime'] ['Action', 'Crime', 'Drama']
8 21bridgesmovie_22225.jpg
count: 3
1.0 ['Action', 'Crime', 'Drama'] ['Action', 'Crime', 'Drama']
9 21bridgesmovie_22226.jpg
count: 3
1.0 ['Drama', 'Crime', 'Action'] ['Action', 'Crime', 'Drama']
10 2

1.0 ['Biography', 'Drama'] ['Biography', 'Drama']
161 abeautifuldaymovie_14556.jpg
count: 2
0.5 ['Romance', 'Drama'] ['Biography', 'Drama']
162 abeautifuldaymovie_14557.jpg
count: 2
1.0 ['Drama', 'Biography'] ['Biography', 'Drama']
163 abeautifuldaymovie_14558.jpg
count: 2
1.0 ['Drama', 'Biography'] ['Biography', 'Drama']
164 abeautifuldaymovie_14559.jpg
count: 2
1.0 ['Drama', 'Biography'] ['Biography', 'Drama']
165 abeautifuldaymovie_14560.jpg
count: 2
1.0 ['Drama', 'Biography'] ['Biography', 'Drama']
166 abeautifuldaymovie_14561.jpg
count: 2
1.0 ['Drama', 'Biography'] ['Biography', 'Drama']
167 abeautifuldaymovie_14562.jpg
count: 2
1.0 ['Drama', 'Biography'] ['Biography', 'Drama']
168 abeautifuldaymovie_14563.jpg
count: 2
1.0 ['Drama', 'Biography'] ['Biography', 'Drama']
169 abeautifuldaymovie_14564.jpg
count: 2
1.0 ['Biography', 'Drama'] ['Biography', 'Drama']
170 abeautifuldaymovie_14565.jpg
count: 2
0.5 ['Drama', 'Comedy'] ['Biography', 'Drama']
171 abeautifuldaymovie_14566.jpg
co

1.0 ['Comedy', 'Animation', 'Adventure'] ['Adventure', 'Animation', 'Comedy']
321 abominablemovie_7326.jpg
count: 3
1.0 ['Adventure', 'Comedy', 'Animation'] ['Adventure', 'Animation', 'Comedy']
322 abominablemovie_7327.jpg
count: 3
1.0 ['Adventure', 'Comedy', 'Animation'] ['Adventure', 'Animation', 'Comedy']
323 abominablemovie_7328.jpg
count: 3
1.0 ['Adventure', 'Comedy', 'Animation'] ['Adventure', 'Animation', 'Comedy']
324 abominablemovie_7329.jpg
count: 3
1.0 ['Adventure', 'Comedy', 'Animation'] ['Adventure', 'Animation', 'Comedy']
325 abominablemovie_7330.jpg
count: 3
1.0 ['Animation', 'Comedy', 'Adventure'] ['Adventure', 'Animation', 'Comedy']
326 abominablemovie_7331.jpg
count: 3
1.0 ['Adventure', 'Comedy', 'Animation'] ['Adventure', 'Animation', 'Comedy']
327 abominablemovie_7332.jpg
count: 3
1.0 ['Comedy', 'Adventure', 'Animation'] ['Adventure', 'Animation', 'Comedy']
328 abominablemovie_7333.jpg
count: 3
1.0 ['Animation', 'Comedy', 'Adventure'] ['Adventure', 'Animation', 'Com

0.6666666666666666 ['Drama', 'Mystery', 'Horror'] ['Adventure', 'Drama', 'Mystery']
480 adastramovie_3339.jpg
count: 3
0.6666666666666666 ['Drama', 'Mystery', 'Thriller'] ['Adventure', 'Drama', 'Mystery']
481 adastramovie_3340.jpg
count: 3
0.6666666666666666 ['Mystery', 'Drama', 'Horror'] ['Adventure', 'Drama', 'Mystery']
482 adastramovie_3341.jpg
count: 3
1.0 ['Adventure', 'Drama', 'Mystery'] ['Adventure', 'Drama', 'Mystery']
483 adastramovie_3342.jpg
count: 3
0.6666666666666666 ['Romance', 'Drama', 'Adventure'] ['Adventure', 'Drama', 'Mystery']
484 adastramovie_3343.jpg
count: 3
1.0 ['Drama', 'Adventure', 'Mystery'] ['Adventure', 'Drama', 'Mystery']
485 adastramovie_3344.jpg
count: 3
1.0 ['Drama', 'Mystery', 'Adventure'] ['Adventure', 'Drama', 'Mystery']
486 adastramovie_3345.jpg
count: 3
0.6666666666666666 ['Drama', 'Adventure', 'Sci-Fi'] ['Adventure', 'Drama', 'Mystery']
487 adastramovie_3346.jpg
count: 3
1.0 ['Drama', 'Mystery', 'Adventure'] ['Adventure', 'Drama', 'Mystery']
488 a

1.0 ['Drama', 'Comedy', 'Adventure'] ['Adventure', 'Comedy', 'Drama']
640 adogsjourneymovie_37698.jpg
count: 3
1.0 ['Comedy', 'Adventure', 'Drama'] ['Adventure', 'Comedy', 'Drama']
641 adogsjourneymovie_37699.jpg
count: 3
0.6666666666666666 ['Fantasy', 'Adventure', 'Drama'] ['Adventure', 'Comedy', 'Drama']
642 adogsjourneymovie_37700.jpg
count: 3
1.0 ['Adventure', 'Drama', 'Comedy'] ['Adventure', 'Comedy', 'Drama']
643 adogsjourneymovie_37701.jpg
count: 3
1.0 ['Drama', 'Adventure', 'Comedy'] ['Adventure', 'Comedy', 'Drama']
644 adogsjourneymovie_37702.jpg
count: 3
1.0 ['Drama', 'Comedy', 'Adventure'] ['Adventure', 'Comedy', 'Drama']
645 adogsjourneymovie_37703.jpg
count: 3
1.0 ['Comedy', 'Drama', 'Adventure'] ['Adventure', 'Comedy', 'Drama']
646 adogsjourneymovie_37704.jpg
count: 3
1.0 ['Drama', 'Comedy', 'Adventure'] ['Adventure', 'Comedy', 'Drama']
647 adogsjourneymovie_37705.jpg
count: 3
1.0 ['Comedy', 'Drama', 'Adventure'] ['Adventure', 'Comedy', 'Drama']
648 adogsjourneymovie_3770

...

1.0 ['Animation', 'Adventure', 'Comedy'] ['Adventure', 'Animation', 'Comedy']
7358 disneyfrozen_34781.jpg
count: 3
1.0 ['Comedy', 'Adventure', 'Animation'] ['Adventure', 'Animation', 'Comedy']
7359 disneyfrozen_34782.jpg
count: 3
1.0 ['Comedy', 'Animation', 'Adventure'] ['Adventure', 'Animation', 'Comedy']
7360 disneyfrozen_34783.jpg
count: 3
1.0 ['Comedy', 'Animation', 'Adventure'] ['Adventure', 'Animation', 'Comedy']
7361 disneyfrozen_34784.jpg
count: 3
1.0 ['Adventure', 'Comedy', 'Animation'] ['Adventure', 'Animation', 'Comedy']
7362 disneyfrozen_34785.jpg
count: 3
1.0 ['Comedy', 'Adventure', 'Animation'] ['Adventure', 'Animation', 'Comedy']
7363 disneyfrozen_34786.jpg
count: 3
1.0 ['Comedy', 'Animation', 'Adventure'] ['Adventure', 'Animation', 'Comedy']
7364 disneyfrozen_34787.jpg
count: 3
1.0 ['Comedy', 'Adventure', 'Animation'] ['Adventure', 'Animation', 'Comedy']
7365 disneyfrozen_34788.jpg
count: 3
1.0 ['Adventure', 'Animation', 'Comedy'] ['Adventure', 'Animation', 'Comedy']
73

0.0 ['Drama', 'Action', 'Sci-Fi'] ['Adventure', 'Family', 'Fantasy']
7514 disneymaleficent_3982.jpg
count: 3
1.0 ['Adventure', 'Fantasy', 'Family'] ['Adventure', 'Family', 'Fantasy']
7515 disneymaleficent_3983.jpg
count: 3
1.0 ['Fantasy', 'Adventure', 'Family'] ['Adventure', 'Family', 'Fantasy']
7516 disneymaleficent_3984.jpg
count: 3
1.0 ['Family', 'Adventure', 'Fantasy'] ['Adventure', 'Family', 'Fantasy']
7517 disneymaleficent_3985.jpg
count: 3
1.0 ['Adventure', 'Family', 'Fantasy'] ['Adventure', 'Family', 'Fantasy']
7518 disneymaleficent_3986.jpg
count: 3
1.0 ['Family', 'Adventure', 'Fantasy'] ['Adventure', 'Family', 'Fantasy']
7519 disneymaleficent_3987.jpg
count: 3
1.0 ['Family', 'Fantasy', 'Adventure'] ['Adventure', 'Family', 'Fantasy']
7520 disneymaleficent_3988.jpg
count: 3
1.0 ['Fantasy', 'Family', 'Adventure'] ['Adventure', 'Family', 'Fantasy']
7521 disneymaleficent_3989.jpg
count: 3
1.0 ['Family', 'Adventure', 'Fantasy'] ['Adventure', 'Family', 'Fantasy']
7522 disneymalefice

7668 doctorsleepmovie_17274.jpg
count: 1
1.0 ['Horror'] ['Horror']
7669 doctorsleepmovie_17275.jpg
count: 1
1.0 ['Horror'] ['Horror']
7670 doctorsleepmovie_17276.jpg
count: 1
1.0 ['Horror'] ['Horror']
7671 doctorsleepmovie_17277.jpg
count: 1
1.0 ['Horror'] ['Horror']
7672 doctorsleepmovie_17278.jpg
count: 1
1.0 ['Horror'] ['Horror']
7673 doctorsleepmovie_17279.jpg
count: 1
1.0 ['Horror'] ['Horror']
7674 doctorsleepmovie_17280.jpg
count: 1
1.0 ['Horror'] ['Horror']
7675 doctorsleepmovie_17281.jpg
count: 1
0.0 ['Drama'] ['Horror']
7676 doctorsleepmovie_17282.jpg
count: 1
1.0 ['Horror'] ['Horror']
7677 doctorsleepmovie_17283.jpg
count: 1
1.0 ['Horror'] ['Horror']
7678 doctorsleepmovie_17284.jpg
count: 1
1.0 ['Horror'] ['Horror']
7679 doctorsleepmovie_17285.jpg
count: 1
0.0 ['Romance'] ['Horror']
7680 doctorsleepmovie_17286.jpg
count: 1
1.0 ['Horror'] ['Horror']
7681 doctorsleepmovie_17287.jpg
count: 1
1.0 ['Horror'] ['Horror']
7682 doctorsleepmovie_17288.jpg
count: 1
1.0 ['Horror'] ['Horr

1.0 ['Drama', 'Comedy', 'Biography'] ['Biography', 'Comedy', 'Drama']
7824 dolemiteismyname_23797.jpg
count: 3
1.0 ['Drama', 'Biography', 'Comedy'] ['Biography', 'Comedy', 'Drama']
7825 dolemiteismyname_23798.jpg
count: 3
1.0 ['Drama', 'Comedy', 'Biography'] ['Biography', 'Comedy', 'Drama']
7826 dolemiteismyname_23799.jpg
count: 3
1.0 ['Drama', 'Comedy', 'Biography'] ['Biography', 'Comedy', 'Drama']
7827 dolemiteismyname_23800.jpg
count: 3
1.0 ['Biography', 'Drama', 'Comedy'] ['Biography', 'Comedy', 'Drama']
7828 dolemiteismyname_23801.jpg
count: 3
1.0 ['Drama', 'Biography', 'Comedy'] ['Biography', 'Comedy', 'Drama']
7829 dolemiteismyname_23802.jpg
count: 3
1.0 ['Biography', 'Drama', 'Comedy'] ['Biography', 'Comedy', 'Drama']
7830 dolemiteismyname_23803.jpg
count: 3
1.0 ['Drama', 'Biography', 'Comedy'] ['Biography', 'Comedy', 'Drama']
7831 dolemiteismyname_23804.jpg
count: 3
1.0 ['Drama', 'Biography', 'Comedy'] ['Biography', 'Comedy', 'Drama']
7832 dolemiteismyname_23805.jpg
count: 3
1

KeyError: 'downtonabbey'

In [25]:
acc_df = pd.DataFrame(li, columns = ['username', 'acc', 'truth', 'top_n'] + columns)
acc_df['acc'].mean()

0.7774633123689726

## 萃取97部電影imdb的feature

### Poster Feature Vector

In [ ]:
test_generator

In [ ]:
extract_layer = Model(inputs=model.input,
                      outputs=model.get_layer('flatten_1').output)
#以这个model的预测值作为输出
extract_output = extract_layer.predict_generator(test_generator,
                                                 steps=STEP_SIZE_TEST,
                                                 verbose=1)
print(extract_output.shape)

In [ ]:
idx = test_generator.filenames
print(len(idx), idx)

In [ ]:
res=pd.DataFrame(extract_output, index = idx)
res

In [ ]:
res.to_csv("./output/PosterFeatureVec.csv", header=False)

### IG images

In [ ]:
DATASET_PATH  = '../IGimg'
train_df = pd.read_csv('../filename2genreMat.csv')
train_df

In [ ]:
train_generator=datagen.flow_from_dataframe(dataframe=train_df,
                                            directory=DATASET_PATH,
                                            x_col="filename",
                                            batch_size=1,
                                            seed=42,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=IMAGE_SIZE)

In [ ]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
print(STEP_SIZE_TRAIN)

In [ ]:
extract_layer = Model(inputs=model.input,
                      outputs=model.get_layer('flatten_1').output)
#以这个model的预测值作为输出
extract_output = extract_layer.predict_generator(train_generator,
                                                 steps=STEP_SIZE_TRAIN,
                                                 verbose=1)
print(extract_output.shape)

In [ ]:
idx = train_generator.filenames
print(len(idx), idx)

In [ ]:
res=pd.DataFrame(extract_output, index = idx)
res

In [ ]:
res.to_csv("./output/IGimgFeatureVec.csv", header=False)